In [18]:
# Import Libraries

import pandas as pd
import numpy as np
from datetime import datetime, date, time

In [19]:
# Open Dataset
clima = pd.read_csv('/home/jovyan/workspace/data/base_clima_parte1.csv', sep = ';')
clima.head()

,data,time,temp,vendas
0,42370,00:00:00,"23,8","14,20923845"
1,42370,01:00:00,23,"13,9582873"
2,42370,02:00:00,"22,5","13,71676794"
3,42370,03:00:00,"22,2","13,63477575"
4,42370,04:00:00,"21,7","13,47386227"


In [20]:
for col in ['vendas','temp']:
    if col == 'vendas':
        clima[col] = clima[col].replace('#VALOR!',np.nan)
    clima[col] = clima[col].str.replace(',','.').astype(float)

## 1.1 Tratar a base de dados de forma que a coluna de data fique no formato YYYY/MM/DD.

In [21]:

data = {}
dataConvertida = []
base_ordinal = date(1899, 12, 30).toordinal()
for day in clima['data'].unique():    
    day = day
    date = date.fromordinal(int(day) + base_ordinal)
    fraction = day - int(day)
    h = int(fraction * 24)
    m = int((fraction*24 - h) * 60)
    s = int((fraction*24*60 - h*60 -m) * 60)
    t = time(h, m, s)

    dtime = datetime.combine(date,t)
    formatted = dtime.strftime('%d/%m/%Y')
    dataConvertida.append(formatted)

data = {timestamp:day for day,timestamp in zip(dataConvertida,clima['data'].unique())}

In [22]:
clima['data'].replace(data, inplace = True)
clima['data'] = clima['data'] + ' ' + clima['time']
clima['data'] = pd.to_datetime(clima['data'])
clima.drop('time',axis = 1, inplace = True)

In [23]:
clima.head()

,data,temp,vendas
0,2016-01-01 00:00:00,23.8,14.209238
1,2016-01-01 01:00:00,23.0,13.958287
2,2016-01-01 02:00:00,22.5,13.716768
3,2016-01-01 03:00:00,22.2,13.634776
4,2016-01-01 04:00:00,21.7,13.473862


## 1.2 Calcule

In [24]:
# Criando variáveis com informações de Dia, Mês e Ano
clima['day'] = clima['data'].dt.day
clima['month'] = clima['data'].dt.month
clima['year'] = clima['data'].dt.year
clima['hour'] = clima['data'].dt.hour
clima['weekday'] = clima['data'].dt.weekday

# Criando variáveis de periodo Manhã, Tarde e Noite
clima['morning'] = clima['hour'].apply(lambda x: 1 if (x >= 6) & (x < 12) else 0)
clima['afternoon'] = clima['hour'].apply(lambda x: 1 if (x >= 12) & (x < 18) else 0)
clima['nigth'] = clima['hour'].apply(lambda x: 1 if (x >= 18) & (x <= 23) else 0)
clima['dawn'] = clima['hour'].apply(lambda x: 1 if (x >= 0) & (x <= 5) else 0)

In [25]:
clima.head()

,data,temp,vendas,day,month,year,hour,weekday,morning,afternoon,nigth,dawn
0,2016-01-01 00:00:00,23.8,14.209238,1,1,2016,0,4,0,0,0,1
1,2016-01-01 01:00:00,23.0,13.958287,1,1,2016,1,4,0,0,0,1
2,2016-01-01 02:00:00,22.5,13.716768,1,1,2016,2,4,0,0,0,1
3,2016-01-01 03:00:00,22.2,13.634776,1,1,2016,3,4,0,0,0,1
4,2016-01-01 04:00:00,21.7,13.473862,1,1,2016,4,4,0,0,0,1


### (a) A temperatura média mensal:

In [26]:
clima['tempMean'] = clima.groupby(['month','year'])['temp'].transform('mean')

### b) Temperatura média no período da manhã, no mês (6hrs ao meio dia)

In [27]:
clima['tempMorningMean'] = clima[clima['morning'] == 1].groupby(['month','year','morning'])['temp'].transform('mean')

### (c) Temperatura média no período da tarde, no mês (meio dia as 18hrs)

In [28]:
clima['tempAfternoonMean'] = clima[clima['afternoon'] == 1].groupby(['month','year','afternoon'])['temp'].transform('mean')

### (d) Temperatura média no período da noite, no mês (18hrs à meia noite)

In [29]:
clima['tempNigthMean'] = clima[clima['nigth'] == 1].groupby(['month','year','nigth'])['temp'].transform('mean')

### (e) Total de vendas (soma) mensal

In [30]:
clima['totalSales'] = clima.groupby(['month','year'])['vendas'].transform('sum')

### (f) Temperatura média no período da madrugada

In [31]:
clima['tempDawnMean'] = clima[clima['dawn'] == 1].groupby(['month','year','dawn'])['temp'].transform('mean')

In [32]:
clima.head()

,data,temp,vendas,day,month,year,hour,weekday,morning,afternoon,nigth,dawn,tempMean,tempMorningMean,tempAfternoonMean,tempNigthMean,totalSales,tempDawnMean
0,2016-01-01 00:00:00,23.8,14.209238,1,1,2016,0,4,0,0,0,1,21.537097,NaN,NaN,NaN,9968.195728,19.84086
1,2016-01-01 01:00:00,23.0,13.958287,1,1,2016,1,4,0,0,0,1,21.537097,NaN,NaN,NaN,9968.195728,19.84086
2,2016-01-01 02:00:00,22.5,13.716768,1,1,2016,2,4,0,0,0,1,21.537097,NaN,NaN,NaN,9968.195728,19.84086
3,2016-01-01 03:00:00,22.2,13.634776,1,1,2016,3,4,0,0,0,1,21.537097,NaN,NaN,NaN,9968.195728,19.84086
4,2016-01-01 04:00:00,21.7,13.473862,1,1,2016,4,4,0,0,0,1,21.537097,NaN,NaN,NaN,9968.195728,19.84086


## 1.3 Salvando em um arquivo CSV

In [33]:
clima.to_csv('/home/jovyan/workspace/data/vendas_sao_paulo.csv', index = False)